## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-23-17-43-00 +0000,wsj,Trump Looks to Chicago Next in Federal Crackdo...,https://www.wsj.com/politics/policy/trump-look...
1,2025-08-23-17-42-42 +0000,nypost,Survey of 77K Iranians finds majority of natio...,https://nypost.com/2025/08/23/world-news/irani...
2,2025-08-23-17-31-25 +0000,nypost,Hurricane Erin destroys dozens of threatened t...,https://nypost.com/2025/08/23/us-news/hurrican...
3,2025-08-23-17-17-02 +0000,nypost,"British Airways flight attendant found naked, ...",https://nypost.com/2025/08/23/world-news/briti...
4,2025-08-23-17-14-25 +0000,nypost,Bus carrying middle school football players to...,https://nypost.com/2025/08/23/us-news/bus-carr...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2379/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,53
311,menendez,17
582,new,15
312,parole,11
615,lyle,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
253,2025-08-22-18-08-00 +0000,wsj,The FBI searched the Maryland home and D.C. of...,https://www.wsj.com/politics/policy/fbi-raids-...,151
51,2025-08-23-12-29-00 +0000,wsj,The Texas Senate approved a redistricting plan...,https://www.wsj.com/politics/policy/texas-cong...,107
39,2025-08-23-13-00-00 +0000,wsj,When President Trump called Gavin Newsom “News...,https://www.wsj.com/politics/policy/to-fight-t...,107
115,2025-08-23-03-00-00 +0000,wsj,Prospective ICE agents are going through a gru...,https://www.wsj.com/politics/policy/we-spent-s...,105
190,2025-08-22-21-03-00 +0000,wsj,Defense Secretary Pete Hegseth fired the Defen...,https://www.wsj.com/politics/national-security...,94


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
253,151,2025-08-22-18-08-00 +0000,wsj,The FBI searched the Maryland home and D.C. of...,https://www.wsj.com/politics/policy/fbi-raids-...
114,74,2025-08-23-03-10-57 +0000,latimes,"Lyle Menendez denied parole, will remain in pr...",https://www.latimes.com/california/story/2025-...
51,54,2025-08-23-12-29-00 +0000,wsj,The Texas Senate approved a redistricting plan...,https://www.wsj.com/politics/policy/texas-cong...
243,52,2025-08-22-18-33-00 +0000,wsj,Canada’s decision to drop tariffs on most U.S....,https://www.wsj.com/articles/canadas-rollback-...
115,46,2025-08-23-03-00-00 +0000,wsj,Prospective ICE agents are going through a gru...,https://www.wsj.com/politics/policy/we-spent-s...
39,40,2025-08-23-13-00-00 +0000,wsj,When President Trump called Gavin Newsom “News...,https://www.wsj.com/politics/policy/to-fight-t...
45,36,2025-08-23-12-42-42 +0000,cbc,U.S. seeks to deport Kilmar Abrego Garcia to U...,https://www.cbc.ca/news/world/kilmar-abrego-ga...
90,36,2025-08-23-07-07-23 +0000,nypost,Family of missing Texas woman Kaura Taylor who...,https://nypost.com/2025/08/23/world-news/texas...
69,33,2025-08-23-09-30-00 +0000,wsj,"At Jackson Hole, Fed Chair Jerome Powell deliv...",https://www.wsj.com/economy/central-banking/po...
214,31,2025-08-22-19-34-00 +0000,wsj,Trump Gives Familiar Two-Week Deadline for Nex...,https://www.wsj.com/world/russia/trump-gives-f...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
